# 01_bronze_ingestion — Camada Bronze (Raw → Delta)

## Objetivo deste notebook
Este notebook realiza a **ingestão do dataset raw**, em csv, para a camada **Bronze**, da arquitetura Medallion.  
A intenção aqui é preservar os dados o mais próximo possível do formato original, com transformações mínimas, garantindo:
- rastreabilidade da origem;
- possibilidade de reprocessamento para Silver/Gold;
- persistência em formato **Delta** para consultas e evoluções posteriores.

## Fonte dos dados
- Arquivo CSV armazenado no Volume Raw do Unity Catalog:
  - `/Volumes/workspace/default/mvp_chess_raw/club_games_dataset_chess_dot_com.csv`
- O dataset contém registros de partidas de xadrez (Chess.com), incluindo metadados.

## Saídas desta etapa
- Tabela Delta Bronze criada/atualizada:
  - `workspace.default.bronze_games`

## Evidências
Este notebook mostra:
- contagem total de registros e colunas;
- schema;
- amostra dos dados (primeiras linhas);
- validação via SQL (COUNT e preview).


In [0]:
file_info = dbutils.fs.ls("/Volumes/workspace/default/mvp_chess_raw")
csv_path = file_info[0].path

df_raw = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("multiLine", "true")
    .option("quote", '"')
    .option("escape", '"')
    .option("mode", "PERMISSIVE")
    .load(csv_path)
)

print(f"Complete File Info: {file_info}")
print(f"CSV Path: {csv_path}")

print()
print(f"Linhas: {df_raw.count()}")
print(f"Colunas: {len(df_raw.columns)}")

print()
df_raw.printSchema()
display(df_raw.limit(10))


In [0]:
bronze_table = "workspace.default.bronze_games"

(
  df_raw.write
  .format("delta")
  .mode("overwrite")
  .saveAsTable(bronze_table)
)

print("Bronze salvo em:", bronze_table)


In [0]:
%sql
SELECT COUNT(*) AS n_rows FROM workspace.default.bronze_games;

In [0]:
%sql
SELECT * FROM workspace.default.bronze_games LIMIT 10;